编写一个用于计算函数耗时的装饰器（参照https://github.com/realpython/materials/blob/master/pandas-fast-flexible-intuitive/tutorial/timer.py） 

In [17]:
import functools
import itertools
import gc
import sys
from timeit import default_timer as _timer

In [18]:
def timeit(_func=None, *, repeat=3, number=1000, file=sys.stdout):
    _repeat = functools.partial(itertools.repeat, None)
    def wrap(func):
        @functools.wraps(func)
        def _timeit(*args, **kwargs):
            gcold = gc.isenabled()
            gc.disable()
            
            try:
                # 外循环：repeat
                trials = []
                for _ in _repeat(repeat):
                    # 内循环  the number of calls within each repeat.
                    total = 0
                    for _ in _repeat(number):
                        start = _timer()
                        result = func(*args, **kwargs)
                        end = _timer()
                        total += end - start
                    trials.append(total)
                
                best = min(trials) / number
                print(
                    "Best of {} trials with {} function"
                    " calls per trial:".format(repeat, number)
                )
                print(
                    "Function `{}` ran in average"
                    " of {:0.3f} seconds.".format(func.__name__, best),
                    end="\n\n",
                    file=file,
                )
                
            finally:
                if gcold:
                    gc.enable()
            # Result is returned *only once*
            return result

        return _timeit

    if _func is None:
        return wrap
    else:
        return wrap(_func)

In [21]:
@timeit
def f():
    return "-".join(str(n) for n in range(100))

f()

Best of 3 trials with 1000 function calls per trial:
Function `f` ran in average of 0.000 seconds.



'0-1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21-22-23-24-25-26-27-28-29-30-31-32-33-34-35-36-37-38-39-40-41-42-43-44-45-46-47-48-49-50-51-52-53-54-55-56-57-58-59-60-61-62-63-64-65-66-67-68-69-70-71-72-73-74-75-76-77-78-79-80-81-82-83-84-85-86-87-88-89-90-91-92-93-94-95-96-97-98-99'

In [22]:
@timeit(number=100000)
def g():
    return "-".join(str(n) for n in range(10))

g()

Best of 3 trials with 100000 function calls per trial:
Function `g` ran in average of 0.000 seconds.



'0-1-2-3-4-5-6-7-8-9'